In [1]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data
spark = get_spark_session("getting_started")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

24/02/26 19:27:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sql("drop table if exists staging.yellow")
spark.sql("drop schema if exists staging")


DataFrame[]

In [ ]:
staging_sql = "CREATE DATABASE staging"
spark.sql(staging_sql)

: 

In [28]:
yellow_taxi_data.write.saveAsTable("staging.yellow_taxi")

Py4JJavaError: An error occurred while calling o132.saveAsTable.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/staging/yellow_taxi/metadata/00003-99ab513d-1edf-42e3-8e6e-c1eb294b3331.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:327)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:300)
	at org.apache.icebe

In [5]:
# if table exists we'll get an error
yellow_taxi_data.write.saveAsTable("staging.yellow_taxi")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `staging`.`yellow_taxi` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

Let's check what we can see in [MinIO console](http://localhost:9001/browser)

In [11]:
!pip install minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
from minio import Minio
client = Minio(
        "172.18.0.3:9000",
        access_key="admin",
        secret_key="password",
        secure=False  # Set to True if your Minio server uses TLS/SSL
    )

In [24]:
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

warehouse 2024-02-26 19:26:16.609000+00:00


In [25]:
def print_objects_in_warehouse(bucket_name):
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        print(obj.object_name)
print_objects_in_warehouse("warehouse")

## Note
As in Spark we can actually specify the mode: Append or Overwrite in order to avoid/mitigate or actually create some sort of solution for our target table

In [29]:
yellow_taxi_data.write.mode("overwrite").saveAsTable("staging.yellow_taxi")

Py4JJavaError: An error occurred while calling o135.saveAsTable.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/staging/yellow_taxi/metadata/00003-99ab513d-1edf-42e3-8e6e-c1eb294b3331.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:327)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:300)
	at org.apache.icebe

## Note 2
But since we're using Iceberg we're getting a bit more!


In [16]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-8c50a032-1042-40dd-9cfe-e6d56d22fcde-00001.parquet
staging/yellow_taxi/data/00000-5-60f4da6f-5faf-4902-a1c1-cb36d985fac2-00001.parquet
staging/yellow_taxi/data/00001-2-8c50a032-1042-40dd-9cfe-e6d56d22fcde-00001.parquet
staging/yellow_taxi/data/00001-6-60f4da6f-5faf-4902-a1c1-cb36d985fac2-00001.parquet
staging/yellow_taxi/metadata/00000-3d75b229-d801-4f96-bbad-6f6a972b3bac.metadata.json
staging/yellow_taxi/metadata/00001-2378b304-7f21-438c-8d4b-97c9107cb0b0.metadata.json
staging/yellow_taxi/metadata/2120078a-925e-42b0-b454-3f72916f54a3-m0.avro
staging/yellow_taxi/metadata/b06c475d-9447-4af4-a415-8128807cf0c4-m0.avro
staging/yellow_taxi/metadata/snap-3736536537737263451-1-b06c475d-9447-4af4-a415-8128807cf0c4.avro
staging/yellow_taxi/metadata/snap-5656612956830299294-1-2120078a-925e-42b0-b454-3f72916f54a3.avro


# Querying Data 

In [17]:
spark.sql("select * from staging.yellow_taxi").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-01-01 00:35:40|  2022-01-01 00:53:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0|    0.5|      3.6

In [18]:
%%sql
select * from staging.yellow_taxi

24/02/26 19:29:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [19]:
%%sql
select count(1) as total_row_count from staging.yellow_taxi

total_row_count
5443362


# Schema Evolution
## *NB* Reusing from Tabular.IO content

In [26]:
%%sql
ALTER TABLE staging.yellow_taxi RENAME COLUMN fare TO fare_amount;


Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/staging/yellow_taxi/metadata/00003-99ab513d-1edf-42e3-8e6e-c1eb294b3331.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:327)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:300)
	at org.apache.iceberg.rest.R

In [27]:
%%sql
ALTER TABLE staging.yellow_taxi RENAME COLUMN fare_amount TO fare;


Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/staging/yellow_taxi/metadata/00003-99ab513d-1edf-42e3-8e6e-c1eb294b3331.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:327)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:300)
	at org.apache.iceberg.rest.R

In [23]:
%%sql
select * from staging.yellow_taxi

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [24]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-7e342c5a-7754-4a77-9c9a-1d98c5a02c0b-00001.parquet
staging/yellow_taxi/data/00000-5-2ba6f1bc-0056-43ca-a74a-8940bb7ba50f-00001.parquet
staging/yellow_taxi/data/00001-2-7e342c5a-7754-4a77-9c9a-1d98c5a02c0b-00001.parquet
staging/yellow_taxi/data/00001-6-2ba6f1bc-0056-43ca-a74a-8940bb7ba50f-00001.parquet
staging/yellow_taxi/metadata/00000-138c209c-21ca-4b7d-a455-8399db6cc246.metadata.json
staging/yellow_taxi/metadata/00001-559da544-0d27-4928-9bc7-41d433cc5857.metadata.json
staging/yellow_taxi/metadata/00002-f112ca18-a7a2-433b-92ff-dea26c60aa1d.metadata.json
staging/yellow_taxi/metadata/00003-f446d40c-bdca-4383-b51a-ee75568bdfe2.metadata.json
staging/yellow_taxi/metadata/00004-2ae6139e-2d9e-485d-a68a-c1b0fcd672c1.metadata.json
staging/yellow_taxi/metadata/562c2f2d-fb38-43a1-b105-792c33682255-m0.avro
staging/yellow_taxi/metadata/bbb4c179-86f7-4660-9d55-8ecfea569d36-m0.avro
staging/yellow_taxi/metadata/snap-1990269890442050925-1-562c2f2d-fb38-43a1-b105-792c33